In [1]:
# setup + import
from utils import *
import os
import sklearn
import pandas as pd
import pickle

# setup + import
data = pd.read_csv('../data_processed/breast-cancer-diagnostic.shuf.lrn.csv')
data.head()

pickle_file = open('../data_processed/breast-cancer_column_types.pkl', 'rb')
feature_structure = pickle.load(pickle_file)
pickle_file.close()

In [2]:
feature_columns = feature_structure['bin'] + feature_structure['cat'] + feature_structure['cont'] + feature_structure['ord']

feature_structure["cat"]
TARGET = feature_structure["target"]


X = data[feature_columns]
y = data[TARGET]

values = [v  for _,v in feature_structure.items()]
# pd.DataFrame({"type":[feature_structure.keys()], "columns":  values} )
pd.DataFrame({"type":feature_structure.keys(), "columns":  values} )

,type,columns
0,bin,[]
1,cat,[]
2,ord,[]
3,cont,"[radiusMean, textureMean, perimeterMean, ar..."
4,target,class


In [6]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

classifiers0 = [
    SVC(kernel='linear', random_state=42, decision_function_shape='ovo', probability=True),
    KNeighborsClassifier(n_neighbors=3, n_jobs=-1),
    DecisionTreeClassifier(random_state=42),
]

classifier1 = [ KNeighborsClassifier(n_jobs=-1)]
model1 = get_pipeline(feature_structure,classifier1[0])

# single model training
res_holdout, model_holdout = perform_holdout(X, y, model1, 42)
res_cv, model_cv = perform_cv(X, y, model1)

results = append_results({}, model_holdout, model_cv, res_holdout, res_cv)
pd.DataFrame(results)

,model,accuracy,precision,recall,f1-score,timing
0,KNeighborsClassifier(n_jobs=-1)_Holdout,0.982456,0.982972,0.982456,0.982400,0.013000
1,KNeighborsClassifier(n_jobs=-1)_CV,0.964912,0.966011,0.964912,0.964515,0.107997


Evaluating multiple models

In [8]:
evaluate_models(data, feature_structure, classifiers0)

,model,accuracy,precision,recall,f1-score,timing
0,"SVC(decision_function_shape='ovo', kernel='lin...",1.000000,1.000000,1.000000,1.000000,0.010004
1,"SVC(decision_function_shape='ovo', kernel='lin...",0.978947,0.979292,0.978947,0.978852,0.069510
2,"KNeighborsClassifier(n_jobs=-1, n_neighbors=3)...",0.982456,0.982972,0.982456,0.982400,0.010000
3,"KNeighborsClassifier(n_jobs=-1, n_neighbors=3)_CV",0.964912,0.965999,0.964912,0.964377,0.075022
4,DecisionTreeClassifier(random_state=42)_Holdout,1.000000,1.000000,1.000000,1.000000,0.012026
5,DecisionTreeClassifier(random_state=42)_CV,0.922807,0.923679,0.922807,0.921534,0.062996
